<a href="https://colab.research.google.com/github/sungkwangsong/EasyOCR/blob/master/notebooks/hiBERT_%EC%97%B0%EA%B5%AC%EC%9E%AC%EB%8B%A8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
HBN_DATASETS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/datasets/hibrainnet'
HBN_MODELS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet'
HBN_EMBEDDINGS_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/embeddings/hibrainnet'
HBN_ARTICLES_DIR_PATH = '/content/drive/MyDrive/Workspaces/Hibrainnet/hbn-data-lake/papers/kci/data'

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

# 시드 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# KoSimCSE-BERT 모델 및 토크나이저 로드
model_name = "BM-K/KoSimCSE-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.to(device)

# 한국연구재단 분류 데이터 로드
def load_nrf_data(file_path, encoding='utf-8'):
    df = pd.read_csv(file_path, encoding=encoding)
    print(f"데이터 로드 완료. 형태: {df.shape}")
    print(f"컬럼: {df.columns.tolist()}")
    return df

# 데이터셋 클래스 정의 (SimCSE 방식)
class NRFDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 각 텍스트를 두 번 토큰화 (dropout을 통한 데이터 증강)
        text = self.texts[idx]

        # 첫 번째 인코딩
        inputs1 = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 두 번째 인코딩 (동일 텍스트, 다른 dropout 마스크)
        inputs2 = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids1': inputs1['input_ids'].squeeze(),
            'attention_mask1': inputs1['attention_mask'].squeeze(),
            'input_ids2': inputs2['input_ids'].squeeze(),
            'attention_mask2': inputs2['attention_mask'].squeeze(),
        }

# SimCSE 손실 함수 정의
class SimCSELoss(torch.nn.Module):
    def __init__(self, temperature=0.05):
        super(SimCSELoss, self).__init__()
        self.temperature = temperature
        self.cos_sim = torch.nn.CosineSimilarity(dim=-1)
        self.loss_fct = torch.nn.CrossEntropyLoss()

    def forward(self, embeddings_a, embeddings_b):
        batch_size = embeddings_a.size(0)

        # 코사인 유사도 계산
        cos_sim_matrix = self.cos_sim(embeddings_a.unsqueeze(1), embeddings_b.unsqueeze(0)) / self.temperature

        # 레이블: 대각선 요소(자기 자신)가 양성 쌍
        labels = torch.arange(batch_size, device=cos_sim_matrix.device)

        return self.loss_fct(cos_sim_matrix, labels)

# 학습 함수
def train(model, data_loader, optimizer, loss_fn, device, epoch):
    model.train()
    total_loss = 0
    valid_batches = 0

    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        try:
            optimizer.zero_grad()

            # 배치 데이터를 GPU로 이동
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)

            # 배치가 비어있는지 확인
            if input_ids1.size(0) == 0 or input_ids2.size(0) == 0:
                continue

            # 첫 번째 문장 임베딩
            outputs1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
            embeddings1 = outputs1.last_hidden_state[:, 0]  # [CLS] 토큰 임베딩

            # 두 번째 문장 임베딩
            outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
            embeddings2 = outputs2.last_hidden_state[:, 0]  # [CLS] 토큰 임베딩

            # 손실 계산
            loss = loss_fn(embeddings1, embeddings2)

            # 역전파
            loss.backward()

            # 그래디언트 클리핑 (폭발 방지)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()
            valid_batches += 1
            progress_bar.set_postfix({'loss': total_loss / (valid_batches if valid_batches > 0 else 1)})

        except Exception as e:
            print(f"배치 처리 중 오류 발생: {e}")
            continue

    return total_loss / (valid_batches if valid_batches > 0 else 1)

# 평가 함수
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            # 배치 데이터를 GPU로 이동
            input_ids1 = batch['input_ids1'].to(device)
            attention_mask1 = batch['attention_mask1'].to(device)
            input_ids2 = batch['input_ids2'].to(device)
            attention_mask2 = batch['attention_mask2'].to(device)

            # 첫 번째 문장 임베딩
            outputs1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
            embeddings1 = outputs1.last_hidden_state[:, 0]

            # 두 번째 문장 임베딩
            outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
            embeddings2 = outputs2.last_hidden_state[:, 0]

            # 손실 계산
            loss = loss_fn(embeddings1, embeddings2)
            total_loss += loss.item()

    return total_loss / len(data_loader)

# 메인 함수
def main():
    # 파일 경로 설정
    file_path = f"{HBN_DATASETS_DIR_PATH}/한국연구재단_전공분류 - 학습용(중분류포함).csv"

    # 데이터 로드
    df = load_nrf_data(file_path, encoding='utf-8')

    # 컬럼명 확인 및 실제 컬럼명으로 접근
    print("정확한 컬럼명:")
    for col in df.columns:
        print(f"- {col}")

    # 학습 데이터 준비
    # 키워드와 설명을 결합하여 텍스트 생성
    texts = []

    # 컬럼명 추출 - 실제 컬럼명과 일치하는지 확인
    keyword_col = [col for col in df.columns if 'KEYWORDS' in col][0]
    description_col = [col for col in df.columns if 'DESCRIPTION' in col][0]

    print(f"사용할 키워드 컬럼: {keyword_col}")
    print(f"사용할 설명 컬럼: {description_col}")

    for _, row in df.iterrows():
        keywords = str(row[keyword_col])
        description = str(row[description_col])

        # 처음 5개 행만 출력하여 데이터 확인
        if _ < 5:
            print(f"행 {_+1} 샘플:")
            print(f"  키워드: {keywords[:100]}...")
            print(f"  설명: {description[:100]}...")

        # 키워드와 설명이 모두 있는 경우만 학습 데이터에 추가
        if keywords != 'nan' and description != 'nan' and keywords.strip() and description.strip():
            # 텍스트 길이가 너무 짧지 않은 경우만 추가 (최소 10자)
            if len(keywords) > 10:
                texts.append(keywords)
            if len(description) > 10:
                texts.append(description)
            # 키워드와 설명을 결합한 텍스트도 추가
            combined = f"{keywords} {description}"
            if len(combined) > 20:  # 결합 텍스트는 더 길어야 함
                texts.append(combined)

    print(f"전처리된 텍스트 수: {len(texts)}")

    # 텍스트 데이터가 너무 적은 경우 경고
    if len(texts) < 100:
        print(f"경고: 학습 데이터가 매우 적습니다 ({len(texts)}개). 결과가 제한적일 수 있습니다.")

        # 데이터 증강: 텍스트 조각화를 통한 추가 데이터 생성
        augmented_texts = []
        for text in texts:
            if len(text) > 50:
                # 긴 텍스트는 여러 조각으로 나누어 추가 데이터 생성
                chunks = [text[i:i+50] for i in range(0, len(text), 25)]  # 50자씩, 25자 겹침
                augmented_texts.extend(chunks)

        texts.extend(augmented_texts)
        print(f"데이터 증강 후 텍스트 수: {len(texts)}")

    # 학습/검증 데이터 분할
    train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=42)

    print(f"학습 데이터 크기: {len(train_texts)}")
    print(f"검증 데이터 크기: {len(val_texts)}")

    # 데이터셋 및 데이터로더 생성
    train_dataset = NRFDataset(train_texts, tokenizer)
    val_dataset = NRFDataset(val_texts, tokenizer)

    # 배치 크기 조정 (데이터셋 크기에 따라)
    batch_size = min(16, max(4, len(train_texts) // 10))  # 데이터셋의 1/10, 최소 4, 최대 16
    print(f"배치 크기: {batch_size}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # 손실 함수 및 옵티마이저 설정
    loss_fn = SimCSELoss(temperature=0.05)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # 학습 실행
    num_epochs = 5
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # 학습
        train_loss = train(model, train_loader, optimizer, loss_fn, device, epoch)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

        # 평가
        val_loss = evaluate(model, val_loader, loss_fn, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        # 최고 성능 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"Saving the best model with validation loss: {val_loss:.4f}")
            model_save_path = f"{HBN_MODELS_DIR_PATH}/hiBERT"
            model.save_pretrained(model_save_path)
            tokenizer.save_pretrained(model_save_path)

    print("학습 완료!")

    # 간단한 테스트
    test_sentence = "인공지능 딥러닝 연구"
    print(f"테스트 문장: '{test_sentence}'")

    # 파인튜닝된 모델로 임베딩 생성
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0].cpu().numpy()

        print(f"임베딩 형태: {embedding.shape}")
        print("임베딩 벡터 (처음 5개 값):", embedding[0][:5])

# 스크립트 실행
if __name__ == "__main__":
    main()

In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import glob

class PaperSearchSystem:
    def __init__(self, model_path, device=None):
        """
        논문 검색 시스템 초기화

        Args:
            model_path: 파인튜닝된 hiBERT 모델 경로
            device: 연산 장치 (None이면 자동 감지)
        """
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        print(f"Using device: {self.device}")

        # 모델 및 토크나이저 로드
        try:
            # 파인튜닝된 모델 로드
            self.tokenizer = BertTokenizer.from_pretrained(model_path)
            self.model = BertModel.from_pretrained(model_path)
            print(f"파인튜닝된 hiBERT 모델을 성공적으로 로드했습니다: {model_path}")
        except Exception as e:
            print(f"모델 로드 중 오류 발생: {e}")
            print(f"기본 KoSimCSE-BERT 모델을 대신 로드합니다.")
            self.tokenizer = BertTokenizer.from_pretrained("BM-K/KoSimCSE-bert")
            self.model = BertModel.from_pretrained("BM-K/KoSimCSE-bert")

        self.model.to(self.device)
        self.model.eval()

        # 논문 데이터 및 임베딩 저장소
        self.papers = []
        self.paper_embeddings = None

    def load_papers_from_dir(self, dir_path):
        """
        디렉토리에서 JSON 논문 파일들을 로드

        Args:
            dir_path: 논문 JSON 파일이 있는 디렉토리 경로
        """
        json_files = glob.glob(os.path.join(dir_path, "*.json"))
        print(f"Found {len(json_files)} JSON files in directory: {dir_path}")

        for file_path in tqdm(json_files, desc="Loading papers"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    paper_data = json.load(f)

                # 기본 값 설정 (파일 구조에 따라 다를 수 있음)
                paper_info = {
                    'title': '',
                    'abstract': '',
                    'keywords': [],
                    'authors': [],
                    'category': '',
                    'file_path': file_path
                }

                # 제목 추출
                article_info = paper_data.get('articleInfo', {})
                title_group = article_info.get('title-group', {})

                if isinstance(title_group.get('article-title'), list):
                    for title_item in title_group.get('article-title', []):
                        if title_item.get('@lang') == 'original':
                            paper_info['title'] = title_item.get('#text', '')
                            break
                elif isinstance(title_group.get('article-title'), dict):
                    paper_info['title'] = title_group.get('article-title', {}).get('#text', '')

                # 초록 추출
                abstract_group = article_info.get('abstract-group', {})
                if isinstance(abstract_group.get('abstract'), list):
                    for abstract_item in abstract_group.get('abstract', []):
                        if abstract_item.get('@lang') == 'original':
                            paper_info['abstract'] = abstract_item.get('#text', '')
                            break
                elif isinstance(abstract_group.get('abstract'), dict):
                    paper_info['abstract'] = abstract_group.get('abstract', {}).get('#text', '')

                # 키워드 추출
                detail_info = paper_data.get('detailInfo', {})
                detail_article_info = detail_info.get('articleInfo', {})
                keyword_group = detail_article_info.get('keyword-group', {})

                if isinstance(keyword_group.get('keyword'), list):
                    paper_info['keywords'] = keyword_group.get('keyword', [])
                elif isinstance(keyword_group.get('keyword'), str):
                    paper_info['keywords'] = [keyword_group.get('keyword', '')]

                # 저자 추출
                author_group = article_info.get('author-group', {})
                if isinstance(author_group.get('author'), list):
                    for author in author_group.get('author', []):
                        if isinstance(author, str):
                            paper_info['authors'].append(author)
                        elif isinstance(author, dict):
                            author_name = author.get('#text', '')
                            if author_name:
                                paper_info['authors'].append(author_name)
                elif isinstance(author_group.get('author'), dict):
                    author_name = author_group.get('author', {}).get('#text', '')
                    if author_name:
                        paper_info['authors'].append(author_name)

                # 카테고리 추출
                paper_info['category'] = article_info.get('article-categories', '')

                # 논문 정보가 제대로 존재하는 경우만 추가
                if paper_info['title'] or paper_info['abstract']:
                    self.papers.append(paper_info)

            except Exception as e:
                print(f"Error loading paper from {file_path}: {e}")

        print(f"Successfully loaded {len(self.papers)} papers.")

    def create_paper_embeddings(self, batch_size=4):
        """
        논문의 임베딩 벡터 생성

        Args:
            batch_size: 배치 크기
        """
        if not self.papers:
            raise ValueError("Please load papers first using load_papers_from_dir()")

        # 제목, 초록, 키워드 결합
        texts = []
        for paper in self.papers:
            title = paper.get('title', '')
            abstract = paper.get('abstract', '')
            keywords = ' '.join(paper.get('keywords', []))

            # 텍스트 결합 (제목이 중요하므로 2번 포함)
            text = f"{title} {title} {abstract} {keywords}"
            texts.append(text)

        # 배치 처리로 임베딩 생성
        embeddings = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Creating paper embeddings"):
            batch_texts = texts[i:i+batch_size]

            # 토큰화 및 텐서 생성
            inputs = self.tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )

            # GPU로 이동
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # 임베딩 생성
            with torch.no_grad():
                outputs = self.model(**inputs)
                batch_embeddings = outputs.last_hidden_state[:, 0].cpu().numpy()
                embeddings.extend(batch_embeddings)

        self.paper_embeddings = np.array(embeddings)
        print(f"Created embeddings with shape: {self.paper_embeddings.shape}")

    def save_embeddings(self, output_file):
        """
        생성된 임베딩 저장

        Args:
            output_file: 저장할 파일 경로
        """
        if self.paper_embeddings is None:
            raise ValueError("No embeddings to save. Create embeddings first.")

        # 논문 정보와 임베딩을 함께 저장
        data_to_save = {
            'papers': self.papers,
            'embeddings': self.paper_embeddings.tolist()
        }

        # 디렉토리 생성
        os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=2)

        print(f"Saved papers and embeddings to {output_file}")

    def load_embeddings(self, embeddings_file):
        """
        저장된 임베딩과 논문 정보 로드

        Args:
            embeddings_file: 임베딩 파일 경로
        """
        with open(embeddings_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        self.papers = data.get('papers', [])
        self.paper_embeddings = np.array(data.get('embeddings', []))

        print(f"Loaded {len(self.papers)} papers and embeddings with shape: {self.paper_embeddings.shape}")

    def search(self, query, top_k=5, min_similarity=0.3):
        """
        쿼리와 의미적으로 유사한 논문 검색

        Args:
            query: 검색 쿼리
            top_k: 반환할 상위 결과 수
            min_similarity: 최소 유사도 점수

        Returns:
            검색 결과 (논문 정보와 유사도 점수)
        """
        if not self.papers or self.paper_embeddings is None:
            raise ValueError("Papers and embeddings must be loaded before searching")

        # 쿼리 임베딩 생성
        inputs = self.tokenizer(
            query,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)
            query_embedding = outputs.last_hidden_state[:, 0].cpu().numpy()

        # 코사인 유사도 계산
        similarities = cosine_similarity(query_embedding, self.paper_embeddings)[0]

        # 유사도 점수에 따라 정렬된 인덱스 가져오기
        sorted_indices = similarities.argsort()[::-1]

        # 결과 생성
        results = []
        for idx in sorted_indices:
            score = similarities[idx]
            if score < min_similarity or len(results) >= top_k:
                break

            paper_info = self.papers[idx].copy()
            paper_info['similarity_score'] = float(score)
            results.append(paper_info)

        return results

def display_search_results(results):
    """
    검색 결과를 화면에 출력

    Args:
        results: 검색 결과 리스트
    """
    if not results:
        print("검색 결과가 없습니다.")
        return

    print(f"\n검색 결과 ({len(results)}개):")
    print("=" * 80)

    for i, paper in enumerate(results):
        print(f"{i+1}. [{paper['similarity_score']:.4f}] {paper['title']}")
        if paper.get('authors'):
            print(f"   저자: {', '.join(paper['authors'])}")
        if paper.get('category'):
            print(f"   분야: {paper['category']}")
        if paper.get('keywords'):
            print(f"   키워드: {', '.join(paper['keywords'])}")
        if paper.get('abstract'):
            # 초록 앞부분만 표시
            abstract = paper['abstract']
            if len(abstract) > 200:
                abstract = abstract[:200] + "..."
            print(f"   초록: {abstract}")
        print(f"   파일: {os.path.basename(paper['file_path'])}")
        print("-" * 80)

def main():
    # 설정 변수들
    model_path = f"{HBN_MODELS_DIR_PATH}/hiBERT"  # 모델 경로
    papers_dir = HBN_ARTICLES_DIR_PATH       # 논문 디렉토리 경로
    embeddings_file = f"{HBN_EMBEDDINGS_DIR_PATH}/articles_embeddings.json"  # 임베딩 파일 경로
    create_embeddings = False                  # 임베딩 새로 생성 여부
    top_k = 5                                  # 검색 결과 개수

    # 테스트용 검색 쿼리들
    test_queries = [
        "온톨로지 기반 검색",
        "인공지능 알고리즘",
        "데이터 마이닝",
        "추론 시스템",
        "지식 그래프"
    ]

    # 검색 시스템 초기화
    search_system = PaperSearchSystem(model_path)

    # 임베딩 생성 또는 로드
    if create_embeddings:
        # 논문 데이터 로드
        search_system.load_papers_from_dir(papers_dir)
        # 임베딩 생성
        search_system.create_paper_embeddings()
        # 임베딩 저장
        search_system.save_embeddings(embeddings_file)
    elif os.path.exists(embeddings_file):
        # 저장된 임베딩 로드
        search_system.load_embeddings(embeddings_file)
    else:
        # 논문 데이터 로드
        search_system.load_papers_from_dir(papers_dir)
        # 임베딩 생성
        search_system.create_paper_embeddings()
        # 임베딩 저장
        search_system.save_embeddings(embeddings_file)

    # 각 테스트 쿼리로 검색 수행
    for query in test_queries:
        print(f"\n[검색어: '{query}']")
        try:
            results = search_system.search(query, top_k=top_k)
            display_search_results(results)
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")

    # 대화형 모드
    print("\n===== hiBERT 한국 논문 검색 시스템 =====")
    print("'quit' 또는 'exit'를 입력하면 종료합니다.\n")

    while True:
        query = input("\n검색어를 입력하세요: ")
        if query.lower() in ['quit', 'exit', '종료']:
            print("검색 시스템을 종료합니다.")
            break

        try:
            results = search_system.search(query, top_k=top_k)
            display_search_results(results)
        except Exception as e:
            print(f"검색 중 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    main()

Using device: cuda
파인튜닝된 hiBERT 모델을 성공적으로 로드했습니다: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
Loaded 698 papers and embeddings with shape: (698, 768)

[검색어: '온톨로지 기반 검색']

검색 결과 (1개):
1. [0.4094] 온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구
   저자: 황현숙(부경대학교), 이준연(동명대학교)
   분야: 전자/정보통신공학
   키워드: Ontology(온톨로지), Association Mining(연관마이닝), Inference Algorithm(추론알고리즘), User Profile Modeling(사용자 프로파일 모델링), Databases(데이터베이스)
   초록: 정보 검색에 대한 연구는 방대한 데이터에서 원하는 검색 정보를 제공할 뿐 만 아니라 개인의 취향에 따른 맞춤 검색 및 추론된 지식을 제공하는 데 초점을 두고 있다. 본 논문의 목적은 데이터를 개념화하여 분류 및 정의할 수 있는 온톨로지 구조를 기반으로 숨어있는 지식을 발견하여 개인 맞춤 검색을 제공하는 추론 알고리즘에 대해 연구하는 것이다. 현재의 검색에서...
   파일: KCI-ART001293627-온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구.json
--------------------------------------------------------------------------------

[검색어: '인공지능 알고리즘']

검색 결과 (2개):
1. [0.3407] 인공지능을 활용하는 행정작용에 대한 법적 고찰
   저자: 강현호(성균관대학교)
   분야: 법학
   키워드: 인공지능(Künstliche Intelligenz), 설명요구권(Recht au

In [3]:
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import glob

def main():
    # 1. 설정 변수 (여기에서 실제 경로로 변경하세요)
    MODEL_PATH = f"{HBN_MODELS_DIR_PATH}/hiBERT"  # hiBERT 모델 경로
    PAPERS_DIR = HBN_ARTICLES_DIR_PATH       # 논문 파일이 있는 디렉토리

    # 사용할 검색어 설정
    QUERIES = [
        "온톨로지 기반 검색 시스템",
        # "데이터 마이닝 기법",
        # "지식 추론 알고리즘",
        # "시맨틱 웹",
        # "연관 마이닝 방법",
        # "SWRL 추론 언어",
        # "Jess 엔진",
        # "온톨로지 제약조건",
        # "개인 맞춤 검색"
    ]

    print(f"[설정 정보]")
    print(f"- 모델 경로: {MODEL_PATH}")
    print(f"- 논문 디렉토리: {PAPERS_DIR}")
    print(f"- 검색어 개수: {len(QUERIES)}개")
    print("-" * 50)

    # 2. 장치 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 장치: {device}")

    # 3. 모델 로드
    try:
        print(f"모델 로드 중...")
        tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
        model = BertModel.from_pretrained(MODEL_PATH)
        model.to(device)
        model.eval()
        print(f"모델 로드 완료: {MODEL_PATH}")
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        print(f"기본 KoSimCSE-BERT 모델로 대체합니다.")
        tokenizer = BertTokenizer.from_pretrained("BM-K/KoSimCSE-bert")
        model = BertModel.from_pretrained("BM-K/KoSimCSE-bert")
        model.to(device)
        model.eval()

    # 4. 논문 파일 로드
    json_files = glob.glob(os.path.join(PAPERS_DIR, "*.json"))
    print(f"논문 파일 {len(json_files)}개 발견")

    papers = []
    for file_path in tqdm(json_files, desc="논문 로드 중"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                paper_data = json.load(f)

            # 논문 정보 추출
            article_info = paper_data.get('articleInfo', {})

            # 제목 추출
            title = ""
            title_group = article_info.get('title-group', {})
            if isinstance(title_group.get('article-title'), list):
                for title_item in title_group.get('article-title', []):
                    if title_item.get('@lang') == 'original':
                        title = title_item.get('#text', '')
                        break
            elif isinstance(title_group.get('article-title'), dict):
                title = title_group.get('article-title', {}).get('#text', '')

            # 초록 추출
            abstract = ""
            abstract_group = article_info.get('abstract-group', {})
            if isinstance(abstract_group.get('abstract'), list):
                for abstract_item in abstract_group.get('abstract', []):
                    if abstract_item.get('@lang') == 'original':
                        abstract = abstract_item.get('#text', '')
                        break
            elif isinstance(abstract_group.get('abstract'), dict):
                abstract = abstract_group.get('abstract', {}).get('#text', '')

            # 논문 정보 저장
            if title or abstract:
                papers.append({
                    'title': title,
                    'abstract': abstract,
                    'file_path': file_path
                })

        except Exception as e:
            print(f"파일 처리 오류 ({file_path}): {e}")

    print(f"총 {len(papers)}개 논문 로드 완료")

    # 5. 임베딩 생성
    print("논문 임베딩 생성 중...")
    paper_embeddings = []

    for paper in tqdm(papers, desc="임베딩 생성"):
        # 제목과 초록 결합 (제목은 더 중요하므로 두 번 포함)
        text = f"{paper['title']} {paper['title']} {paper['abstract']}"

        # 토큰화
        inputs = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        # GPU로 이동
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 임베딩 계산
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0].cpu().numpy()
            paper_embeddings.append(embedding[0])

    paper_embeddings = np.array(paper_embeddings)
    print(f"임베딩 생성 완료: {paper_embeddings.shape}")

    # 6. 각 검색어에 대한 검색 수행
    for i, query in enumerate(QUERIES, 1):
        print(f"\n[{i}/{len(QUERIES)}] 검색어: '{query}'")

        # 쿼리 임베딩 생성
        inputs = tokenizer(
            query,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            query_embedding = outputs.last_hidden_state[:, 0].cpu().numpy()

        # 유사도 계산
        similarities = cosine_similarity(query_embedding, paper_embeddings)[0]
        top_indices = similarities.argsort()[::-1][:5]  # 상위 5개 결과

        # 결과 출력
        print("-" * 80)
        for rank, idx in enumerate(top_indices, 1):
            paper = papers[idx]
            score = similarities[idx]
            print(f"{rank}. [{score:.4f}] {paper['title']}")

            # 초록 앞부분 표시
            abstract = paper['abstract']
            if len(abstract) > 200:
                abstract = abstract[:200] + "..."
            print(f"   초록: {abstract}")
            print(f"   파일: {os.path.basename(paper['file_path'])}")
            print("-" * 80)

    print("\n모든 검색 테스트가 완료되었습니다.")

if __name__ == "__main__":
    main()

[설정 정보]
- 모델 경로: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
- 논문 디렉토리: /content/drive/MyDrive/Workspaces/Hibrainnet/hbn-data-lake/papers/kci/data
- 검색어 개수: 1개
--------------------------------------------------
사용 장치: cuda
모델 로드 중...
모델 로드 완료: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
논문 파일 717개 발견


논문 로드 중: 100%|██████████| 717/717 [00:13<00:00, 53.02it/s] 


총 717개 논문 로드 완료
논문 임베딩 생성 중...


임베딩 생성: 100%|██████████| 717/717 [00:09<00:00, 79.16it/s]

임베딩 생성 완료: (717, 768)

[1/1] 검색어: '온톨로지 기반 검색 시스템'
--------------------------------------------------------------------------------
1. [0.4261] 온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구
   초록: 정보 검색에 대한 연구는 방대한 데이터에서 원하는 검색 정보를 제공할 뿐 만 아니라 개인의 취향에 따른 맞춤 검색 및 추론된 지식을 제공하는 데 초점을 두고 있다. 본 논문의 목적은 데이터를 개념화하여 분류 및 정의할 수 있는 온톨로지 구조를 기반으로 숨어있는 지식을 발견하여 개인 맞춤 검색을 제공하는 추론 알고리즘에 대해 연구하는 것이다. 현재의 검색에서...
   파일: KCI-ART001293627-온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구.json
--------------------------------------------------------------------------------
2. [0.2350] 웹 기반의 삼차원 스테레오 카메라 원격제어 시스템 설계 및 구현
   초록: 
   파일: KCI-ART000910284-웹 기반의 삼차원 스테레오 카메라 원격제어 시스템 설계 및 구현.json
--------------------------------------------------------------------------------
3. [0.2316] 나노간극 구동기를 이용한 나노기계적 단백질 검출기
   초록: 
   파일: KCI-ART001094584-나노간극 구동기를 이용한 나노기계적 단백질 검출기.json
-----------------------------------

In [4]:
"""
논문 검색 모듈: hiBERT 모델을 사용한 의미 기반 논문 검색 기능 제공
"""

import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import glob

class PaperSearchEngine:
    """
    hiBERT 기반 논문 검색 엔진 클래스
    """

    def __init__(self, model_path):
        """
        검색 엔진 초기화

        Args:
            model_path (str): hiBERT 모델 경로
        """
        self.model_path = model_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.papers = []
        self.paper_embeddings = None
        self.load_model()

    def load_model(self):
        """모델 및 토크나이저 로드"""
        try:
            self.tokenizer = BertTokenizer.from_pretrained(self.model_path)
            self.model = BertModel.from_pretrained(self.model_path)
            self.model.to(self.device)
            self.model.eval()
            print(f"hiBERT 모델 로드 성공: {self.model_path}")
        except Exception as e:
            print(f"모델 로드 실패: {e}")
            print("기본 KoSimCSE-BERT 모델로 대체합니다.")
            self.tokenizer = BertTokenizer.from_pretrained("BM-K/KoSimCSE-bert")
            self.model = BertModel.from_pretrained("BM-K/KoSimCSE-bert")
            self.model.to(self.device)
            self.model.eval()

    def load_papers_from_dir(self, papers_dir):
        """
        디렉토리에서 논문 파일 로드

        Args:
            papers_dir (str): 논문 JSON 파일 디렉토리 경로

        Returns:
            int: 로드된 논문 수
        """
        json_files = glob.glob(os.path.join(papers_dir, "*.json"))
        print(f"논문 파일 {len(json_files)}개 발견")

        self.papers = []
        for file_path in tqdm(json_files, desc="논문 로드 중"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    paper_data = json.load(f)

                paper_info = self._extract_paper_info(paper_data, file_path)

                # 논문 정보가 제대로 존재하는 경우만 추가
                if paper_info['title'] or paper_info['abstract']:
                    self.papers.append(paper_info)

            except Exception as e:
                print(f"파일 처리 오류 ({file_path}): {e}")

        print(f"총 {len(self.papers)}개 논문 로드 완료")
        return len(self.papers)

    def _extract_paper_info(self, paper_data, file_path):
        """
        논문 데이터에서 필요한 정보 추출

        Args:
            paper_data (dict): 논문 JSON 데이터
            file_path (str): 파일 경로

        Returns:
            dict: 추출된 논문 정보
        """
        # 기본 정보 구조 생성
        paper_info = {
            'title': '',
            'abstract': '',
            'keywords': [],
            'authors': [],
            'category': '',
            'file_path': file_path
        }

        # 논문 정보 추출
        article_info = paper_data.get('articleInfo', {})
        detail_info = paper_data.get('detailInfo', {})
        detail_article_info = detail_info.get('articleInfo', {})

        # 제목 추출
        title_group = article_info.get('title-group', {})
        if isinstance(title_group.get('article-title'), list):
            for title_item in title_group.get('article-title', []):
                if title_item.get('@lang') == 'original':
                    paper_info['title'] = title_item.get('#text', '')
                    break
        elif isinstance(title_group.get('article-title'), dict):
            paper_info['title'] = title_group.get('article-title', {}).get('#text', '')

        # 초록 추출
        abstract_group = article_info.get('abstract-group', {})
        if isinstance(abstract_group.get('abstract'), list):
            for abstract_item in abstract_group.get('abstract', []):
                if abstract_item.get('@lang') == 'original':
                    paper_info['abstract'] = abstract_item.get('#text', '')
                    break
        elif isinstance(abstract_group.get('abstract'), dict):
            paper_info['abstract'] = abstract_group.get('abstract', {}).get('#text', '')

        # 키워드 추출
        keyword_group = detail_article_info.get('keyword-group', {})
        if isinstance(keyword_group.get('keyword'), list):
            paper_info['keywords'] = keyword_group.get('keyword', [])
        elif isinstance(keyword_group.get('keyword'), str):
            paper_info['keywords'] = [keyword_group.get('keyword', '')]

        # 저자 추출
        author_group = article_info.get('author-group', {})
        if isinstance(author_group.get('author'), list):
            for author in author_group.get('author', []):
                if isinstance(author, str):
                    paper_info['authors'].append(author)
                elif isinstance(author, dict):
                    author_name = author.get('#text', '')
                    if author_name:
                        paper_info['authors'].append(author_name)
        elif isinstance(author_group.get('author'), dict):
            author_name = author_group.get('author', {}).get('#text', '')
            if author_name:
                paper_info['authors'].append(author_name)

        # 카테고리 추출
        paper_info['category'] = article_info.get('article-categories', '')

        return paper_info

    def create_embeddings(self, batch_size=4):
        """
        논문 임베딩 생성

        Args:
            batch_size (int): 배치 크기

        Returns:
            numpy.ndarray: 생성된 임베딩 배열
        """
        if not self.papers:
            raise ValueError("논문을 먼저 로드해야 합니다.")

        paper_embeddings = []

        for paper in tqdm(self.papers, desc="임베딩 생성"):
            # 제목, 키워드, 초록 결합
            title = paper.get('title', '')
            abstract = paper.get('abstract', '')
            keywords_str = ' '.join(paper.get('keywords', []))

            # 결합 텍스트 생성 (제목과 키워드에 가중치 부여)
            combined_text = f"{title} {title} {keywords_str} {keywords_str} {abstract}"

            # 임베딩 생성
            embedding = self._create_embedding(combined_text)
            paper_embeddings.append(embedding)

        self.paper_embeddings = np.array(paper_embeddings)
        print(f"임베딩 생성 완료: {self.paper_embeddings.shape}")
        return self.paper_embeddings

    def _create_embedding(self, text):
        """
        텍스트에 대한 임베딩 생성

        Args:
            text (str): 임베딩할 텍스트

        Returns:
            numpy.ndarray: 생성된 임베딩 벡터
        """
        # 토큰화
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        # GPU로 이동
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # 임베딩 계산
        with torch.no_grad():
            outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state[:, 0].cpu().numpy()[0]

        return embedding

    def search(self, query, top_k=5):
        """
        쿼리로 논문 검색

        Args:
            query (str): 검색 쿼리
            top_k (int): 반환할 상위 결과 수

        Returns:
            list: 검색 결과 (논문 정보와 유사도 점수)
        """
        if not self.papers or self.paper_embeddings is None:
            raise ValueError("논문과 임베딩을 먼저 생성해야 합니다.")

        # 쿼리 임베딩 생성
        query_embedding = self._create_embedding(query)

        # 유사도 계산
        similarities = cosine_similarity([query_embedding], self.paper_embeddings)[0]

        # 상위 결과 인덱스
        top_indices = similarities.argsort()[::-1][:top_k]

        # 결과 생성
        results = []
        for idx in top_indices:
            paper_info = self.papers[idx].copy()
            paper_info['similarity_score'] = float(similarities[idx])
            results.append(paper_info)

        return results

    def get_paper_similarity_matrix(self, max_papers=None):
        """
        논문 간 유사도 매트릭스 계산

        Args:
            max_papers (int, optional): 계산할 최대 논문 수

        Returns:
            tuple: (유사도 매트릭스, 대상 논문 목록)
        """
        if self.paper_embeddings is None:
            raise ValueError("임베딩을 먼저 생성해야 합니다.")

        # 계산할 논문 수 제한 (메모리 문제 방지)
        if max_papers is not None and max_papers < len(self.papers):
            embeddings = self.paper_embeddings[:max_papers]
            papers = self.papers[:max_papers]
        else:
            embeddings = self.paper_embeddings
            papers = self.papers

        # 유사도 매트릭스 계산
        similarity_matrix = cosine_similarity(embeddings)

        return similarity_matrix, papers

    def save_embeddings(self, output_file):
        """
        임베딩 저장

        Args:
            output_file (str): 저장할 파일 경로
        """
        if self.paper_embeddings is None:
            raise ValueError("임베딩을 먼저 생성해야 합니다.")

        # 저장할 데이터 준비
        data = {
            'papers': self.papers,
            'embeddings': self.paper_embeddings.tolist()
        }

        # 디렉토리 생성
        os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)

        # 파일에 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"임베딩 저장 완료: {output_file}")

    def load_embeddings(self, input_file):
        """
        저장된 임베딩 로드

        Args:
            input_file (str): 임베딩 파일 경로

        Returns:
            bool: 성공 여부
        """
        try:
            with open(input_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            self.papers = data.get('papers', [])
            self.paper_embeddings = np.array(data.get('embeddings', []))

            print(f"임베딩 로드 완료: {input_file}")
            print(f"논문 {len(self.papers)}개, 임베딩 형태 {self.paper_embeddings.shape}")
            return True

        except Exception as e:
            print(f"임베딩 로드 실패: {e}")
            return False

In [5]:
"""
논문 검색 모듈 테스트 스크립트
"""

import os
import time

def print_search_result(result, rank):
    """
    검색 결과 출력 헬퍼 함수

    Args:
        result (dict): 검색 결과 항목
        rank (int): 순위
    """
    print(f"{rank}. [{result['similarity_score']:.4f}] {result['title']}")

    # 저자 표시
    if result.get('authors'):
        authors_str = ', '.join(result['authors'])
        print(f"   저자: {authors_str}")

    # 키워드 표시
    if result.get('keywords'):
        keywords_str = ', '.join(result['keywords'])
        print(f"   키워드: {keywords_str}")

    # 카테고리 표시
    if result.get('category'):
        print(f"   분야: {result['category']}")

    # 초록 앞부분 표시
    abstract = result.get('abstract', '')
    if len(abstract) > 200:
        abstract = abstract[:200] + "..."
    print(f"   초록: {abstract}")
    print(f"   파일: {os.path.basename(result['file_path'])}")
    print("-" * 80)

def print_similarity_matrix(similarity_matrix, papers, max_display=10):
    """
    유사도 매트릭스 출력

    Args:
        similarity_matrix: 유사도 매트릭스
        papers: 논문 목록
        max_display: 최대 표시할 행/열 수
    """
    n = min(similarity_matrix.shape[0], max_display)

    print("\n논문 간 유사도 매트릭스:")
    print("-" * 80)

    # 헤더 출력
    header = "ID |"
    for i in range(n):
        paper_id = f"P{i+1}"
        header += f" {paper_id:^6} |"
    print(header)
    print("-" * len(header))

    # 유사도 매트릭스 출력
    for i in range(n):
        row = f"P{i+1} |"
        for j in range(n):
            if i == j:
                row += f" {'---':^6} |"  # 자기 자신과의 유사도는 표시 생략
            else:
                row += f" {similarity_matrix[i, j]:.4f} |"
        print(row)

    print("-" * len(header))

    # 각 논문 제목 출력
    for i in range(n):
        print(f"P{i+1}: {papers[i]['title'][:50]}{'...' if len(papers[i]['title']) > 50 else ''}")

def test_search_engine():
    """
    논문 검색 엔진 테스트 함수
    """
    # 설정 변수
    MODEL_PATH = f"{HBN_MODELS_DIR_PATH}/hiBERT"  # hiBERT 모델 경로
    PAPERS_DIR = HBN_ARTICLES_DIR_PATH       # 논문 디렉토리 경로
    EMBEDDINGS_FILE = f"{HBN_EMBEDDINGS_DIR_PATH}/paper_embeddings.json"  # 임베딩 저장/로드 파일
    CREATE_NEW_EMBEDDINGS = False              # 새 임베딩 생성 여부

    # 테스트 검색어
    TEST_QUERIES = [
        "온톨로지 기반 검색 시스템",
        "데이터 마이닝 기법",
        "지식 추론 알고리즘",
        # "시맨틱 웹",
        # "연관 마이닝 방법",
        # "SWRL 추론 언어",
        # "Jess 엔진",
        # "온톨로지 제약조건",
        # "개인 맞춤 검색"
    ]

    # 시작 시간 기록
    start_time = time.time()

    print(f"[테스트 설정]")
    print(f"- 모델 경로: {MODEL_PATH}")
    print(f"- 논문 디렉토리: {PAPERS_DIR}")
    print(f"- 임베딩 파일: {EMBEDDINGS_FILE}")
    print(f"- 새 임베딩 생성: {CREATE_NEW_EMBEDDINGS}")
    print(f"- 테스트 검색어: {len(TEST_QUERIES)}개")
    print("-" * 80)

    # 1. 검색 엔진 초기화
    search_engine = PaperSearchEngine(MODEL_PATH)

    # 2. 임베딩 로드 또는 생성
    if not CREATE_NEW_EMBEDDINGS and os.path.exists(EMBEDDINGS_FILE):
        # 기존 임베딩 로드
        success = search_engine.load_embeddings(EMBEDDINGS_FILE)
        if not success:
            print("임베딩 로드 실패, 새로 생성합니다.")
            CREATE_NEW_EMBEDDINGS = True

    if CREATE_NEW_EMBEDDINGS or not os.path.exists(EMBEDDINGS_FILE):
        # 새 임베딩 생성
        search_engine.load_papers_from_dir(PAPERS_DIR)
        search_engine.create_embeddings()
        search_engine.save_embeddings(EMBEDDINGS_FILE)

    # 3. 각 테스트 검색어로 검색 실행
    print("\n===== 검색 테스트 시작 =====")

    for i, query in enumerate(TEST_QUERIES, 1):
        print(f"\n[{i}/{len(TEST_QUERIES)}] 검색어: '{query}'")

        try:
            # 검색 수행
            results = search_engine.search(query, top_k=5)

            # 결과 출력
            print("-" * 80)
            for rank, result in enumerate(results, 1):
                print_search_result(result, rank)

        except Exception as e:
            print(f"검색 중 오류 발생: {e}")

    # 4. 논문 간 유사도 분석
    try:
        similarity_matrix, papers = search_engine.get_paper_similarity_matrix(max_papers=10)
        print_similarity_matrix(similarity_matrix, papers)
    except Exception as e:
        print(f"유사도 분석 중 오류 발생: {e}")

    # 5. 실행 시간 출력
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\n테스트 완료! 실행 시간: {elapsed_time:.2f}초")

if __name__ == "__main__":
    test_search_engine()

[테스트 설정]
- 모델 경로: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
- 논문 디렉토리: /content/drive/MyDrive/Workspaces/Hibrainnet/hbn-data-lake/papers/kci/data
- 임베딩 파일: /content/drive/MyDrive/Workspaces/Hibrainnet/embeddings/hibrainnet/paper_embeddings.json
- 새 임베딩 생성: False
- 테스트 검색어: 3개
--------------------------------------------------------------------------------
hiBERT 모델 로드 성공: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
임베딩 로드 완료: /content/drive/MyDrive/Workspaces/Hibrainnet/embeddings/hibrainnet/paper_embeddings.json
논문 698개, 임베딩 형태 (698, 768)

===== 검색 테스트 시작 =====

[1/3] 검색어: '온톨로지 기반 검색 시스템'
--------------------------------------------------------------------------------
1. [0.3659] 온톨로지 기반에서 연관 마이닝 방법을 이용한 지식 추론 알고리즘 연구
   저자: 황현숙(부경대학교), 이준연(동명대학교)
   키워드: Ontology(온톨로지), Association Mining(연관마이닝), Inference Algorithm(추론알고리즘), User Profile Modeling(사용자 프로파일 모델링), Databases(데이터베이스)
   분야: 전자/정보통신공학
   초록: 정보 검색에 대한 연구는 방대한 데이터에서 원하는

In [12]:
"""
논문 검색 모듈 간단 사용 예제
특정 검색어로 논문 검색 및 결과 출력
"""

import os

# 설정 변수 (실제 환경에 맞게 수정)
MODEL_PATH = f"{HBN_MODELS_DIR_PATH}/hiBERT"     # hiBERT 모델 경로
PAPERS_DIR = HBN_ARTICLES_DIR_PATH          # 논문 디렉토리 경로
EMBEDDINGS_FILE = f"{HBN_EMBEDDINGS_DIR_PATH}/paper_embeddings.json"     # 임베딩 파일 경로

# 사용할 검색어 정의
# QUERY = "온톨로지 기반 연관 마이닝 방법을 이용한 지식 추론"
# QUERY = "다문화 가정"
QUERY = "도시 재생"

def main():
    """
    메인 함수: 검색 모듈 사용 예제
    """
    print(f"검색어: '{QUERY}'")
    print("-" * 60)

    # 1. 검색 엔진 초기화
    engine = PaperSearchEngine(MODEL_PATH)

    # 2. 임베딩 로드 또는 생성
    if os.path.exists(EMBEDDINGS_FILE):
        # 기존 임베딩 로드
        engine.load_embeddings(EMBEDDINGS_FILE)
    else:
        # 새 임베딩 생성
        engine.load_papers_from_dir(PAPERS_DIR)
        engine.create_embeddings()
        engine.save_embeddings(EMBEDDINGS_FILE)

    # 3. 검색 실행
    results = engine.search(QUERY, top_k=3)

    # 4. 결과 출력
    print(f"\n검색 결과 ({len(results)}개):")
    for i, paper in enumerate(results, 1):
        print(f"\n결과 {i}:")
        print(f"제목: {paper['title']}")
        print(f"유사도: {paper['similarity_score']:.4f}")

        if paper.get('keywords'):
            print(f"키워드: {', '.join(paper['keywords'])}")

        if paper.get('abstract'):
            # 초록 앞부분 표시
            abstract = paper['abstract']
            if len(abstract) > 150:
                abstract = abstract[:150] + "..."
            print(f"초록: {abstract}")

        print("-" * 60)

if __name__ == "__main__":
    main()

검색어: '도시 재생'
------------------------------------------------------------
hiBERT 모델 로드 성공: /content/drive/MyDrive/Workspaces/Hibrainnet/models/hibrainnet/hiBERT
임베딩 로드 완료: /content/drive/MyDrive/Workspaces/Hibrainnet/embeddings/hibrainnet/paper_embeddings.json
논문 698개, 임베딩 형태 (698, 768)

검색 결과 (3개):

결과 1:
제목: 광주광역시의 도시 재생과 지속가능한 도시 성장 방안
유사도: 0.3618
키워드: 도시 재생, 지속가능한 도시 성장, 어반 폴리, 주택재개발, 주택재건축., urban regeneration, sustainable urban growth, urban folly, housing redevelopment, housing reconstruction.
초록: 전 세계적으로 최근의 도시정책은 구도심 쇠퇴와 도시 재생이 쟁점이다. 본 연구에서는 광주광역시의 도시 재생과정을 고찰하고, 이를 기반으로 도시 정체성과 잠재자원에 기초한 지속가능한 도시성장 방안을 모색하고자 하였다. 분석 결과 광주시 도시 재생은 1960년대 이후 재개...
------------------------------------------------------------

결과 2:
제목: 미국 텍사스주 러벅시의 도시 구조와 지속가능한 도시 재생
유사도: 0.3491
키워드: urban structure, civic center declining, sustainable regeneration, culture, 도시 구조, 도심 쇠퇴, 지속가능한 도시 재생, 문화
초록: 최근에는 도시의 환경적, 경제적, 사회적 조화를 주제로 하는 문화·역사·정체성·생태 중심의 지속가능한 재생이 다양하게 제시되고 있다. 하지만, 도시 재생의 기반이 되는 도시 구조는 배제되어